# ALeRCE watchlists, list of oids

```Author: Alejandra Muñoz Arancibia```

ALeRCE notebook

Read a list of object identifiers, connect to the database and check which of the objects have recent detections (date criterion can be changed). Save the list of objects that meet the date criterion to a csv file, including their information from the ``objects`` table

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
from astropy.time import Time
import psycopg2
from datetime import date, datetime, timedelta
import json

Connect to DB

In [3]:
dir_dbfile = '/home/ammunoz1/Desktop/Astro2020/ALeRCE/alercebroker/usecases/'
credentials_file = dir_dbfile+'alercereaduser_v2.json' #online data
#credentials_file = dir_dbfile+'alercereaduser_v3.json' #new db

with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'],
                        password=params['password'])

Read file with oids (csv format, first column has oid, comments start with '#')

In [4]:
dir_watchlists = '/home/ammunoz1/Desktop/Astro2020/ALeRCE/Watchlists/'
namefil = dir_watchlists+'sources.csv'

In [5]:
objs = pd.read_csv(namefil, header=None, sep=',', comment='#', skip_blank_lines=True)
objs.rename(columns={0: 'oid'}, inplace=True)

if len(objs.columns)>1:
    objs = objs[['oid']]

#display(objs.head())
print('Number of oids:', len(objs))

objs_uniq = pd.unique(objs['oid'])
objs = pd.DataFrame(objs_uniq, columns=['oid'])
nobjs = len(objs)

display(objs.head())
print('Number of unique oids:', nobjs)

Number of oids: 52


,oid
0,ZTF20abpgnos
1,ZTF20abwezjk
2,ZTF20abxgxjd
3,ZTF20abyptpc
4,ZTF18abqyaix


Number of unique oids: 52


Check that all oids are found in the database

In [6]:
query = '''
SELECT
    *
FROM
    objects
WHERE
    oid in (%s)
''' % ",".join(["'%s'" % oid for oid in objs['oid']])
objs_indb = pd.read_sql_query(query, conn)
objs_indb.set_index('oid', inplace=True)
nobjs_indb = len(objs_indb)

display(objs_indb.head())
print('Number of oids found in db:', nobjs_indb)

if nobjs_indb!=nobjs:
    print('Number of unique oids:', nobjs, 'Problem!')

,nobs,mean_magap_g,mean_magap_r,median_magap_g,median_magap_r,max_magap_g,max_magap_r,min_magap_g,min_magap_r,sigma_magap_g,...,deltajd,lastmjd,firstmjd,period,catalogid,classxmatch,classrf,pclassrf,pclassearly,classearly
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18abqyaix,46,18.112278,18.199050,18.133901,18.184250,18.661301,18.735500,17.444599,17.690500,0.284267,...,21.978912,59095.397998,59073.419086,None,None,None,10.0,0.298800,0.621764,19
ZTF18acsvogv,12,17.886529,17.598240,17.721001,17.526899,19.176600,18.102900,17.108400,16.962601,0.826194,...,25.963704,59145.129005,59119.165301,None,None,None,11.0,0.268500,0.461903,19
ZTF19abyvqes,7,19.799999,19.892500,19.799999,19.751801,19.899599,20.494600,19.700399,19.488701,0.140855,...,3.037685,59118.446238,59115.408553,None,None,None,NaN,NaN,0.503165,19
ZTF20aafqxxd,5,NaN,17.027880,NaN,16.892799,NaN,18.277800,NaN,16.073500,NaN,...,0.014005,59110.471794,59110.457789,None,None,None,NaN,NaN,0.366620,19
ZTF20aatxryt,44,18.947260,18.874132,19.050900,19.021700,19.282801,20.254101,17.958000,17.302299,0.327757,...,253.316528,59168.528715,58915.212188,None,None,None,14.0,0.337824,0.530542,19


Number of oids found in db: 52


Find current datetime in local and UTC formats

In [7]:
date_now_local = datetime.now()
offset_utc = 3.
date_now_utc = date_now_local+timedelta(hours=offset_utc)

date_now_local = date_now_local.strftime('%Y-%m-%dT%H:%M:%S')
date_now_utc = date_now_utc.strftime('%Y-%m-%dT%H:%M:%S')

date_now_local, date_now_utc

('2020-11-16T16:24:34', '2020-11-16T19:24:34')

Define date criterion. Example: last 3 days (in MJD)

In [8]:
date_now_utc = Time(date_now_utc, format='isot', scale='utc')

date_lim = Time(date_now_utc, format="isot", scale="utc").mjd
date_lim = date_lim-3.

date_lim

59166.80872685185

Apply date criterion

In [9]:
query = '''
SELECT
    *
FROM
    objects
WHERE
    lastmjd >= %s
    AND oid in (%s)
''' % (date_lim, ','.join(["'%s'" % oid for oid in objs['oid']]))
objs_indb = pd.read_sql_query(query, conn)
objs_indb.set_index('oid', inplace=True)

display(objs_indb.head())
print('Number of oids found in db that match criterion:', len(objs_indb))

,nobs,mean_magap_g,mean_magap_r,median_magap_g,median_magap_r,max_magap_g,max_magap_r,min_magap_g,min_magap_r,sigma_magap_g,...,deltajd,lastmjd,firstmjd,period,catalogid,classxmatch,classrf,pclassrf,pclassearly,classearly
oid,,,,,,,,,,,,,,,,,,,,,
ZTF20aatxryt,44,18.947260,18.874132,19.050900,19.021700,19.282801,20.254101,17.958000,17.302299,0.327757,...,253.316528,59168.528715,58915.212188,None,None,None,14.0,0.337824,0.530542,19
ZTF20abyptpc,68,15.802083,15.652770,15.329800,15.359200,17.652100,17.818800,14.616600,14.634500,1.130153,...,70.867002,59169.227546,59098.360544,None,None,None,12.0,0.256736,0.628658,19
ZTF20abywydb,43,19.510526,19.261917,19.533701,19.092050,20.781200,20.974899,18.500900,18.398300,0.682794,...,63.866296,59168.290590,59104.424294,None,None,None,14.0,0.330672,0.495561,19
ZTF20acfucwr,35,18.568018,18.798083,18.662100,19.126349,20.113001,20.636200,17.008600,17.112900,0.931531,...,46.845532,59168.140231,59121.294699,None,None,None,10.0,0.326024,0.449320,19
ZTF20acgided,39,17.465700,17.623635,17.419000,17.422800,17.791901,19.425800,17.164101,17.178801,0.195136,...,45.916157,59169.217176,59123.301018,None,None,None,14.0,0.269280,0.593914,19


Number of oids found in db that match criterion: 10


Save to csv file

In [10]:
namein = namefil.replace(dir_watchlists, '')
namein = namein.replace('.csv', '_out_')

namefil2 = dir_watchlists+'out/'+namein+str(date_now_utc).replace(':', '_')+'.csv'
objs_indb.to_csv(namefil2)

print('Saved to file:', namefil2)

Saved to file: /home/ammunoz1/Desktop/Astro2020/ALeRCE/Watchlists/out/sources_out_2020-11-16T19_24_34.000.csv


Close connection to DB

In [11]:
conn.close()